<a href="https://colab.research.google.com/github/gsasikiran/asus_combined/blob/sasi/asus_combined_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate


In [38]:
data = ImageDataBunch.from_folder('asus_combined/asus_combined_dataset', igno)


/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:415: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:418: UserWarning: Your validation set is empty. If this is by design, use `no_split()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


IndexError: ignored

asus_combined/asus_combined_dataset/training_mobilenet_v2_035_096.txt
